In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!pip install scipy==1.0.0
!pip install pillow==4.1.1
!pip install fastai

    100% |████████████████████████████████| 50.0MB 649kB/s 
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1


In [0]:
from fastai.imports import*
from fastai.structured import*
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd

In [0]:
pd.options.display.max_rows = 1000

In [8]:
!pwd

/content


In [0]:
import os
os.chdir('/content/drive/AI/Santander')

In [0]:
df_train = pd.read_csv('train.csv', low_memory = False)
df_test = pd.read_csv('test.csv', low_memory = False)

In [12]:
df_train.shape

(4459, 4993)

In [13]:
df_test.shape

(49342, 4992)

In [47]:
df_train.head(3)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [0]:
train_Id = df_train['ID']
train_Target = df_train['target']
test_Id = df_test['ID']

In [25]:
dtype_df = df_train.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,3147
1,float64,1845
2,object,1


In [28]:
unique_df = df_train.nunique().reset_index()
unique_df.columns = ['Col_name', 'Unique_count']
constant_df = unique_df[unique_df['Unique_count']==1]
constant_df.shape


(256, 2)

In [0]:
df_train.drop(constant_df.Col_name, axis =1, inplace = True)
df_test.drop(constant_df.Col_name, axis = 1, inplace = True)

In [0]:
model = RandomForestRegressor(random_state =42, n_jobs =-1)

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
param_grid = {
    'max_depth': [80, 90, 100, 110],
    'max_features': [0.5 , 'sqrt', 'log2'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 500]
}

In [0]:
predictors = [x for x in df_train.columns if x not in ['ID' ,'target']]

In [52]:
predictors

['48df886f9',
 '0deb4b6a8',
 '34b15f335',
 'a8cb14b00',
 '2f0771a37',
 '30347e683',
 'd08d1fbe3',
 '6ee66e115',
 '20aa07010',
 'dc5a8f1d8',
 '11d86fa6a',
 '77c9823f2',
 '8d6c2a0b2',
 '4681de4fd',
 'adf119b9a',
 'cff75dd09',
 '96f83a237',
 'b8a716ebf',
 '6c7a4567c',
 '4fcfd2b4d',
 'f3b9c0b95',
 '71cebf11c',
 'd966ac62c',
 '68b647452',
 'c88d108c9',
 'ff7b471cd',
 '0d866c3d7',
 'bc3f77679',
 'bd8f989f1',
 '0eff5bf95',
 '22ed6dba3',
 '92b13ebba',
 '233c7c17c',
 '2cb4d123e',
 '87ffda550',
 '822e49b95',
 '316b978cd',
 'd04e16aed',
 '5d5c5ce6d',
 'ec863cb52',
 '11ad148bd',
 'ea18d720e',
 '408d86ce9',
 '69d8b4020',
 'b0868a049',
 '80b14398e',
 'ecdef52b2',
 'c36bcacc5',
 '151d318cd',
 'ab0cddb90',
 '2d6bd8275',
 'dfdf4b580',
 'c87722678',
 'a8c320153',
 'a04684f1f',
 'fa977f17b',
 'feed9d437',
 '645b47cde',
 '7298ca1ef',
 'd80abf8bc',
 '654dd8a3b',
 '7fc39583c',
 '8c94b6675',
 'e421c414e',
 '964335fdf',
 '5a86cabd0',
 '6cb207ac9',
 'c1bc828da',
 'e90ed19da',
 'c4d38135f',
 '968b01ca3',
 '26b4

In [53]:
CV_RF = RandomizedSearchCV(estimator = model ,  param_distributions = param_grid, cv =5, verbose =1)
CV_RF.fit(df_train[predictors], train_Target)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 134.6min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'max_depth': [80, 90, 100, 110], 'max_features': [0.5, 'sqrt', 'log2'], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 200, 300, 500]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [54]:
CV_RF.best_params_

{'max_depth': 110,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 300}

In [0]:
import pickle

In [0]:
Rf = RandomForestRegressor(random_state= 42, max_depth= 110, max_features= 'sqrt', min_samples_leaf= 4, min_samples_split =8, n_estimators= 300, n_jobs = -1)


In [0]:
filename = 'TunedRF.sav'
pickle.dump(Rf, open(filename, 'wb'))

In [0]:
filename1 = 'RandomizedsearchRF.sav'
pickle.dump(CV_RF, open(filename1, 'wb'))

In [60]:
Rf.fit(df_train[predictors], train_Target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=8,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [62]:
df_test.head()

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
predictions = Rf.predict(df_test[predictors])

In [0]:
submission = pd.DataFrame({'ID': test_Id, 'target': predictions})

In [0]:
submission.to_csv('RF(fastai).csv', index = False)